In [8]:
from Crypto.Util.number import *

def construct_poly_mul_mat(n, v, b):
    assert v[-1] == 1  # use this after monic

    # step1 construct a matrix of d=a*b
    mat1 = Matrix(ZZ, n, 2 * n - 1)
    for i in range(n):
        for j in range(n):
            mat1[i, j + i] = b[j]

    # step2 construct a matrix of c=d%v
    mat2 = Matrix(ZZ, 2 * n - 1, n)
    for i in range(n):
        mat2[i, i] = 1
    for i in range(n, 2 * n - 1):
        for j in range(i - n, n):
            mat2[i, j] = -v[j - (i - n)]

        init_row = vector(ZZ, n * [0])
        for j in range(i - n):
            temp = -v[n - 1 - j] * vector(ZZ, mat2[i - j - 1])
            init_row += temp
        for j in range(n):
            mat2[i, j] += init_row[j]

    # step3 multiply them and return
    return mat1 * mat2

out = [50, 32, 83, 12, 49, 34, 81, 101, 46, 108, 106, 57, 105, 115, 102, 51, 67, 34, 124, 15, 125, 117, 51, 124, 38, 10, 30, 76, 125, 27, 89, 14, 50, 93, 88, 56]
out = [i<<57 for i in out]
set_random_seed(1337)
p = 6143872265871328074704442651454311068421530353607832481181
pp = 18315300953692143461
Fp = GF(p)
a, b = Fp.random_element(), Fp.random_element()
c = [out[3*i:3*i+3]for i in range(len(out)//3)]
C = [Fp(i) for i in c]
m = Fp.modulus().list()
A = [a]
for i in range(10):
    A.append(A[-1]*a)
AA = []
for i in A:
    AA.append(construct_poly_mul_mat(3,m,i.list()))
B = [a*C[0]+b-C[1]]
for i in range(10):
    B.append(a*B[-1]+a*C[i+1]+b-C[i+2])
BB=[]
for i in B:
    BB.extend(i)
M = zero_matrix(ZZ, 37)
I = identity_matrix(3)
for i in range(len(BB)):
    M[i,i]=pp
    M[-1,i]=BB[i]
for i in range(len(AA)+1):
    if i<len(AA):
        M[-4,3*i:3*(i+1)]=AA[i][0]
        M[-3,3*i:3*(i+1)]=AA[i][1]
        M[-2,3*i:3*(i+1)]=AA[i][2]
    else:
        M[-4,3*i:3*(i+1)]=I[0]
        M[-3,3*i:3*(i+1)]=I[1]
        M[-2,3*i:3*(i+1)]=I[2]
M[-1,-1] = 2**64
L=M.LLL()
for L1 in L:
    l=L1[-4:-1]
    s1=[]
    for i,j in zip(l,out[0:3]):
        s1.append(i+j)
    s1=Fp(s1)
    s0=(s1-b)*a^(-1)
    s0=s0.to_integer()
    print(long_to_bytes(int(s0)))

NameError: name 'long_to_bytes' is not defined

In [17]:
from Crypto.Util.number import *
from gmpy2 import *
from tqdm import *

p = 324556397741108806830285502585098109678766437252172614832253074632331911859471735318636292671562523
c = 141624663734155235543198856069652171779130720945875442624943917912062658275440028763836569215230250

prefix = b"NSSCTF{"
suffix = b"}"
length = 88 - len(prefix) - len(suffix)

#part1 remove prefix and suffix
c -= 256^(len(suffix) + length) * bytes_to_long(prefix)
c -= bytes_to_long(suffix)
c = c * inverse(256,p) % p

L = Matrix(ZZ,length+2,length+2)
for i in range(length):
    L[i,i] = 1
    L[i,-1] = 256^i
    c -= 256^i*48
    c -= 256^i*3

L[-2,-2] = 1
L[-2,-1] = -c
L[-1,-1] = p
L[:,-1:] *= p
res = L.BKZ()
flag = ""
for i in res[:-1]:
    # print(i)
    if(all(abs(j) < 8 for j in i[:-2])):
        print(i[-2])
        for j in i[:-2][::-1]:
            flag += chr(51 - j)
        print(flag)

-1
25350625451533421162474265547571536103420331260232652121722452361537257541460235


25350625451533421162474265547571536103420331260232652121722452361537257541460235
